# Batch Pipeline

In [1]:
import hopsworks

project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://35.171.129.215/p/119
Connected. Call `.close()` to terminate connection gracefully.


## Data Query
JDBC storage connector is not functioning in Python engine. Here I create an abstraction function to connect to a postgres and perform any SQL query a user want via `psycopg2`.

In [2]:
import pandas as pd
import psycopg2

host="localhost"
database="hopsworks"
user="postgres"
password="password"
port=5432

def postgres_query(query):
    with psycopg2.connect("host='{}' port={} dbname='{}' user={} password={}".format(host, port, database, user, password)) as conn:
        data = pd.read_sql_query(query, conn)
    return data

In [3]:
df = postgres_query(query="select * from marketplace_engagement_events;")

In [4]:
df

,user_id,partner,label,event,event_timestamp
0,1,Safe drivers can save up to 40%,TopLevel,view,2023-04-12
1,1,LoanInATapCard,TopLevel,click,2023-04-25
2,1,LoanInATapCard,TopLevel,view,2023-05-06
3,1,Safe drivers can save up to 40%,TopLevel,view,2023-04-29
4,1,"Borrow up to $10,000",TopLevel,view,2023-05-04
5,1,Safe drivers can save up to 40%,TopLevel,view,2023-05-07
6,1,It’s never too late to go back to school,TopLevel,view,2023-05-10
7,1,"Borrow up to $10,000",TopLevel,view,2023-05-09
8,2,Safe drivers can save up to 40%,TopLevel,view,2023-04-12
9,2,LoanInATapCard,Travel,click,2023-04-25


## Data Cleaning / Transformation

In [5]:
df["event_day"] = df['event_timestamp'].values.astype('<M8[M]')
df = df.drop(['event_timestamp'], axis=1)
df

,user_id,partner,label,event,event_day
0,1,Safe drivers can save up to 40%,TopLevel,view,2023-04-01
1,1,LoanInATapCard,TopLevel,click,2023-04-01
2,1,LoanInATapCard,TopLevel,view,2023-05-01
3,1,Safe drivers can save up to 40%,TopLevel,view,2023-04-01
4,1,"Borrow up to $10,000",TopLevel,view,2023-05-01
5,1,Safe drivers can save up to 40%,TopLevel,view,2023-05-01
6,1,It’s never too late to go back to school,TopLevel,view,2023-05-01
7,1,"Borrow up to $10,000",TopLevel,view,2023-05-01
8,2,Safe drivers can save up to 40%,TopLevel,view,2023-04-01
9,2,LoanInATapCard,Travel,click,2023-04-01


## Aggregation

In [6]:
user_label_event = df.groupby(["user_id", "label", "event", "event_day"]).agg({"event": "count"}).rename(columns={"event": "count"}).reset_index()
user_label_event

,user_id,label,event,event_day,count
0,1,TopLevel,click,2023-04-01,1
1,1,TopLevel,view,2023-04-01,2
2,1,TopLevel,view,2023-05-01,5
3,2,TopLevel,view,2023-04-01,1
4,2,TopLevel,view,2023-05-01,3
5,2,Travel,click,2023-04-01,1
6,2,Travel,view,2023-04-01,1
7,2,Travel,view,2023-05-01,2


In [7]:
user_label_event_count = fs.get_or_create_feature_group(
    name="user_label_event_count",
    version=1,
    description="Count per month per event per label per user",
    primary_key=['user_id', 'label', 'event'],
    event_time='event_day',
    online_enabled=True,
    stream=True
)

user_label_event_count.insert(user_label_event)

Feature Group created successfully, explore it at 
https://35.171.129.215/p/119/fs/67/fg/13


Uploading Dataframe: 0.00% |          | Rows 0/8 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://35.171.129.215/p/119/jobs/named/user_label_event_count_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fbb5030d370>, None)

## Second layer aggregation

In [8]:
label_event = user_label_event.groupby(["label", "event", "event_day"]).agg({"count": "sum"}).reset_index()
label_event

,label,event,event_day,count
0,TopLevel,click,2023-04-01,1
1,TopLevel,view,2023-04-01,3
2,TopLevel,view,2023-05-01,8
3,Travel,click,2023-04-01,1
4,Travel,view,2023-04-01,1
5,Travel,view,2023-05-01,2


In [9]:
label_event_count = fs.get_or_create_feature_group(
    name="label_event_count",
    version=1,
    description="Count per month per event per label",
    primary_key=['label', 'event'],
    event_time='event_day',
    online_enabled=True,
    stream=True
)

label_event_count.insert(label_event)

Feature Group created successfully, explore it at 
https://35.171.129.215/p/119/fs/67/fg/14


Uploading Dataframe: 0.00% |          | Rows 0/6 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://35.171.129.215/p/119/jobs/named/label_event_count_1_offline_fg_backfill/executions


(<hsfs.core.job.Job at 0x7fbb503068b0>, None)